In [1]:
import numpy as np

import src

In [2]:
train_models = src.shapenet.get_shapenet_data("train")
model_path = train_models[np.random.randint(len(train_models))]
cloud = src.shapenet.sample_points(model_path, nb=100000)
extractor = src.dataset.PPFPatchExtractor(nb_patches=2044, nb_points=1024)
ppf, M = extractor.make_example(cloud)

In [4]:
import tensorflow as tf
import tensorflow.keras as keras

class PPFNet:
    def make_model(self):
        inputs = keras.Input(shape=(None,None,10)) # (batch, patches, points_per_patches, ppf_features)
        x = inputs 
        
        #Patch pointNet
        for units in self.point_net_units[:-1]:
            x = keras.layers.Dense(units)(x)
            if self.use_batch_norm:
                x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation(keras.activations.relu)(x)
        x = keras.layers.Dense(self.point_net_units[-1])(x)
        x = tf.reduce_max(x, axis=-2)# (batch, patches, patche_features)
        local_features = x
        
        #Computing global features, nothing in the loop for orinigal implementation
        for i,units in enumerate(self.global_units):
            x = keras.layers.Dense(units)(x)
            if i != len(self.global_units) -1:
                if self.use_batch_norm:
                    x = keras.layers.BatchNormalization()(x)
                x = keras.layers.Activation(keras.activations.relu)(x)            
        x = tf.reduce_max(x, axis=-2)# (batch, patches)
        
        #Combining features
        x = tf.repeat(x[...,tf.newaxis,:], tf.shape(local_features)[-2], axis=-2)
        x = tf.concat([local_features, x], axis=-1)
        
        #Computing combined features
        for units in self.mlp_units[:-1]: 
            x = keras.layers.Dense(units)(x)
            if self.use_batch_norm:
                x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation(keras.activations.relu)(x)
        x = keras.layers.Dense(self.point_net_units[-1])(x)
        
        return keras.models.Model(inputs, x)
    
    def __init__(self, 
                 point_net_units=[32,32,32], 
                 mlp_units=[64,64], 
                 global_units=[],
                 use_batch_norm=True,
                ):
        self.point_net_units = point_net_units
        self.mlp_units = mlp_units
        self.global_units = global_units
        self.use_batch_norm = use_batch_norm
        
        self.model = self.make_model()
        
ppfnet = PPFNet()
pred = ppfnet.model(ppf[np.newaxis])

In [5]:
pred

<tf.Tensor: shape=(1, 2044, 32), dtype=float32, numpy=
array([[[ 2.01860443e-02,  7.44291723e-01, -3.64723921e-01, ...,
         -1.99749798e-01,  1.50939807e-01, -6.23417869e-02],
        [ 2.26361752e-02,  7.63214111e-01, -4.45456684e-01, ...,
         -2.61724353e-01,  1.39588580e-01, -8.60113651e-02],
        [-5.05266190e-02,  6.76984310e-01, -4.56612110e-01, ...,
         -1.68286577e-01,  1.07052416e-01,  1.33529305e-04],
        ...,
        [-3.28571051e-02,  7.36261725e-01, -4.44855630e-01, ...,
         -2.20516697e-01,  1.02040432e-01, -4.55468893e-04],
        [-4.38063890e-02,  7.00815022e-01, -4.66150403e-01, ...,
         -2.08308220e-01,  5.46253063e-02, -4.52954769e-02],
        [-8.95734057e-02,  6.93334103e-01, -3.63862991e-01, ...,
         -1.96622208e-01,  9.99273136e-02, -2.80843042e-02]]],
      dtype=float32)>